# Exercice 5 
> Wrap the output of the second exercise in a web service that returns the data in JSON format 


In [50]:
import pandas as pd
from flask import Flask
import os.path

bookings = "./challenge/bookings.csv.bz2"
db_tops = "./DBTops.csv"

In [51]:
def create_DBTops():
    print("Creating Tops DB")
    sum_df = pd.DataFrame()

    for chunk in pd.read_csv(bookings,sep='^', usecols=['arr_port','pax','year' ],chunksize = 100000):  
        chunk=chunk[chunk['year']==2013]
        chunk=chunk[['arr_port','pax']]
        chunk['arr_port']=chunk['arr_port'].str.strip().str.upper()            
        df_aux=chunk.groupby('arr_port').sum()
        sum_df=sum_df.append(df_aux)

    top_airports=sum_df.groupby('arr_port').sum() \
                        .sort_values(by='pax',ascending=False) \
                        .applymap(lambda number: '%d' % number)
    
    top_airports.to_csv(db_tops, index=True, index_label='Arrival airport', sep = '^')

def getTopN(top_n):
    top_airports = pd.read_csv(db_tops,sep='^', nrows = top_n)
    return top_airports.head(top_n).to_json()  



In [52]:
app = Flask(__name__)

@app.route('/top/<int:top_n>', methods=['GET'])
def search_name(top_n):
    top_airports = getTopN(top_n)
    return top_airports

if __name__ == '__main__':
    if (not os.path.isfile(db_tops)):
        create_DBTops()
    else: 
        print ('Empezamos teniendo BD Tops creada')
    app.run(debug=False, port=5000)

Empezamos teniendo BD Tops creada
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [03/Feb/2021 09:38:28] "GET /top/5 HTTP/1.1" 200 -
127.0.0.1 - - [03/Feb/2021 09:38:33] "GET /top/10 HTTP/1.1" 200 -
127.0.0.1 - - [03/Feb/2021 09:38:38] "GET /top/5 HTTP/1.1" 200 -
127.0.0.1 - - [03/Feb/2021 09:38:43] "GET /top/10 HTTP/1.1" 200 -
